In the current notebook, the customer dimension is **managed as SCD1**, meaning it always reflects only the latest state of each customer’s attributes. In bigger systems chances are, address would be a different dimension.

SCD 1 approach for a dimension that's likely to change somewhat frequently creates  subtle but critical flaws as we will see in the FICA logic as well .

In South Africa's financial sector, FICA (Financial Intelligence Centre Act) compliance is the frontline defense against all kinds of finance related illegal activities. 

We will make an assumption that financial institutions monitor customer transactions against geographic risk profiles up to street level.

Our current system uses **SCD1 (Slowly Changing Dimension Type 1)** which simply overwrites old addresses with new ones. 

Creating a temporal blind spot in compliance monitoring. 


When our protagonist customer **Jommy Dubal** moves from a low-risk street(**Techvona**) to a high-risk one (**Vleys Wonderline**), we lose the historical context needed to properly evaluate his past transactions.

The result? We might miss genuine compliance violations or trigger false alerts that waste investigative resources. 

In the current notebook, our customer dimension reflects only the latest state of each customer's attributes, creating subtle but critical flaws in our simplified FICA logic.

The question becomes: "Was this customer actually living in a high-risk area when they made that large deposit a while back?" 

**SCD1 can't answer this - but SCD2 can** transform our compliance monitoring from guesswork into improved intelligence.


This alert logic screams "deadline pressure" and not so clear requirements - a junior engineer  nailed the basic joins, code reviews wouldn't hurt. The core logic works: link deposits to addresses, check against FICA thresholds, flag violations. Job done, right?

Not quite. The output  excludes the actual addresses, making it impossible to debug why alerts fired. 

Munior dev treats the customer dimension like it's frozen in time ,awareness of SCD concepts would help.

The result? Code that works beautifully until someone changes address, then either misses genuine compliance violations or flags innocent transactions based on where customers live now rather than where they lived when making deposits. 

Kind of "works on my machine" solution.

In [ ]:
# Jommy makes a deposit 
fact_deposit_2024_01_10 = pd.DataFrame({
    'deposit_id': [1],
    'customer_id': [1001],
    'deposit_date': ['2024-01-10'],
    'deposit_amount': [250000]
})

alert_logic(fact_deposit_2024_01_10, dim_customer, dim_static_fica_alert)

Dev uses the function and yay The alert is correct: Jommy’s deposit is below the threshold for Techvona Rd. SCD1 appears to work.

In [ ]:
# Jommy moves to Vleyz Wonderline (lower threshold)
dim_customer.loc[0, ['street_name', 'street_code', 'last_modified']] = ['Vleyz Wonderline St', 654, '2025-07-01']

Then what we worried about happens, Jommy downgrades to Vleys and the dimension table keeps its number of rows while overwriting street history, nobody told our junior dev,oppps.

In [ ]:
# Jommy makes a new deposit after the move
fact_deposit_2025_07_10 = pd.DataFrame({
    'deposit_id': [2],
    'customer_id': [1001],
    'deposit_date': ['2025-07-10'],
    'deposit_amount': [150000]
})

alert_logic(fact_deposit_2025_07_10, dim_customer, dim_static_fica_alert)

Junior's function worked again, promotion loading

In [ ]:
alert_logic(fact_deposit_2024_01_10, dim_customer, dim_static_fica_alert)

Disaster strikes when junior was raving about how well the alert method worked, all of a sudden the deposit which was correct from first execution has triggered an alert. 

What happened ? 

For starters the threshold amount shrunk like Jommy's downgrade.

Late night investigation looms for our dev

##### Now let's embark to an alternative universe of SCD2

In [ ]:
dim_customer_scd2 = pd.DataFrame({
    'customer_id': [1001],
    'name': ['Jommy Dubal'],
    'street_name': ['Techvona Rd'],
    'street_code': [5],
    'last_modified': '2024-01-05',
    'valid_from': '2024-01-05',
    'valid_to': None,
    'is_current': 1
})

Now our dimension table in our dimension table we sport 2 columns whose names are similar

**valid_from** and **valid_to** ,what's that about ?

then **is_curren**t too ? 

Busy dimension  but **governance friendly**

- **valid_from** now tracks when we first knew about the address
- **valid_to** provides **audit trails** for historical analysis so **when Jommy moves we will update this value**
- **is_current** ensures **query performance** as we are gong to lean on filtering by this column when our focus is current record, and at the same time data lineage has improved

In [ ]:
# New data to append

dim_customer_scd2.loc[0, ['valid_to','is_current']] = ['2025-07-10',None ]

dim_customer_scd2_new_address = pd.DataFrame({
    'customer_id': [1001],
    'name': ['Jommy Dubal'],
    'street_name': ['Vleys Wonderline St'],
    'street_code': [564],
    'last_modified': '2025-07-10',
    'valid_from': '2024-07-11',
    'valid_to': None,
    'is_current': 1
})

# Append using concat
result = pd.concat([dim_customer_scd2, dim_customer_scd2_new_address], ignore_index=True)
display(result)

Now when our junor tries his flawed method, at least things are not that bad

In [ ]:
alert_logic(fact_deposit_2024_01_10, result, dim_static_fica_alert)

Junior better do a code review to improve that method but SCD2 partially saved the day